In [1]:
## 01.
# 고객의 신상정보 데이터를 통한 회사 서비스 이탈 예측

In [1]:
import pandas as pd

X = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_train.csv")
y = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_test.csv")

X = X.drop(columns=['Surname'])
x_test = x_test.drop(columns=['Surname'])

X['Gender'] = X['Gender'].replace(' male', 'Male').replace('female', 'Female')
x_test['Gender'] = x_test['Gender'].replace(' male', 'Male').replace('female', 'Female')
X['Gender'] = X['Gender'].replace('Male', 0).replace('Female', 1)
x_test['Gender'] = x_test['Gender'].replace('Male', 0).replace('Female', 1)

Geo_dummies = pd.get_dummies(X['Geography'], drop_first=True)
X = X.drop(columns=['Geography'])

x_test_cust_id = x_test['CustomerId']
X = X.drop(columns=['CustomerId'])
x_test = x_test.drop(columns=['CustomerId'])
y = y.drop(columns=['CustomerId'])

X = pd.concat([X, Geo_dummies], axis=1)

Geo_test_dummies = pd.get_dummies(x_test['Geography'], drop_first=True)
x_test = x_test.drop(columns=['Geography'])

x_test = pd.concat([x_test, Geo_test_dummies], axis=1)

data = pd.concat([X, y], axis=1)

##

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# X['Balance'] = scaler.fit_transform(X[['Balance']])
X['EstimatedSalary'] = scaler.fit_transform(X[['EstimatedSalary']])

x_test['Balance'] = scaler.transform(x_test[['Balance']])
x_test['EstimatedSalary'] = scaler.transform(x_test[['EstimatedSalary']])

# X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
# x_test = pd.DataFrame(scaler.transform(x_test), columns=x_test.columns)

##

from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=10)

##

from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

model_xgb = XGBClassifier(n_estimators=200, max_depth=3, eval_metric='error', random_state=10)
model_xgb.fit(x_train, y_train)
y_val_pred = pd.DataFrame(model_xgb.predict(x_val))
print(roc_auc_score(y_val, y_val_pred))


from sklearn.ensemble import RandomForestClassifier

model_rfc = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=13, random_state=10)
model_rfc.fit(x_train, y_train)
y_val_pred2 = pd.DataFrame(model_rfc.predict(x_val))
print(roc_auc_score(y_val, y_val_pred2))

y_test_pred = pd.DataFrame(model_xgb.predict(x_test)).rename(columns={0:'Exited'})

final = pd.concat([x_test_cust_id, y_test_pred], axis=1)
# final.to_csv('12345.csv', index=False)
final

/Users/dabeen/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/dabeen/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0.7223244337001816


<ipython-input-1-64c28ab8be4b>:67: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_rfc.fit(x_train, y_train)


0.7151605406560531


,CustomerId,Exited
0,15601012,1
1,15734762,1
2,15586757,0
3,15590888,0
4,15726087,1
...,...,...
3496,15733966,1
3497,15669994,0
3498,15712403,1
3499,15643819,0


In [2]:
## 02. 
# 이직여부 판단 데이터 (target: 1: 이직 , 0 : 이직 x)

In [3]:
import pandas as pd

X = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_train.csv")
y = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_test.csv")

X['gender'] = X['gender'].fillna('Other')
x_test['gender'] = x_test['gender'].fillna('Other')
X['enrolled_university'] = X['enrolled_university'].fillna('no_enrollment')
x_test['enrolled_university'] = x_test['enrolled_university'].fillna('no_enrollment')
X['education_level'] = X['education_level'].fillna('no school')
x_test['education_level'] = x_test['education_level'].fillna('no school')
X['major_discipline'] = X['major_discipline'].fillna('No Major')
x_test['major_discipline'] = x_test['major_discipline'].fillna('No Major')
X['experience'] = X['experience'].fillna(0)
X['experience'] = X['experience'].replace('<1', 0).replace('>20', 21)
x_test['experience'] = x_test['experience'].fillna(0)
x_test['experience'] = x_test['experience'].replace('<1', 0).replace('>20', 21)

X['company_size'] = X['company_size'].fillna('<10')
x_test['company_size'] = x_test['company_size'].fillna('<10')
X['experience'] = X['experience'].astype(int)
x_test['experience'] = x_test['experience'].astype(int)
X['company_type'] = X['company_type'].fillna('Other')
x_test['company_type'] = x_test['company_type'].fillna('Other')
X['last_new_job'] = X['last_new_job'].fillna('never')
x_test['last_new_job'] = x_test['last_new_job'].fillna('never')
X['relevent_experience'] = X['relevent_experience'].replace('No relevent experience', 0).replace('Has relevent experience', 1)
x_test['relevent_experience'] = x_test['relevent_experience'].replace('No relevent experience', 0).replace('Has relevent experience', 1)

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

X['gender'] = encoder.fit_transform(X['gender'])
X['enrolled_university'] = encoder.fit_transform(X['enrolled_university'])
X['education_level'] = encoder.fit_transform(X['education_level'])
X['major_discipline'] = encoder.fit_transform(X['major_discipline'])
X['company_size'] = encoder.fit_transform(X['company_size'])
X['company_type'] = encoder.fit_transform(X['company_type'])
X['last_new_job'] = encoder.fit_transform(X['last_new_job'])

x_test['gender'] = encoder.fit_transform(x_test['gender'])
x_test['enrolled_university'] = encoder.fit_transform(x_test['enrolled_university'])
x_test['education_level'] = encoder.fit_transform(x_test['education_level'])
x_test['major_discipline'] = encoder.fit_transform(x_test['major_discipline'])
x_test['company_size'] = encoder.fit_transform(x_test['company_size'])
x_test['company_type'] = encoder.fit_transform(x_test['company_type'])
x_test['last_new_job'] = encoder.fit_transform(x_test['last_new_job'])

x_test_id = x_test['enrollee_id']

X = X.drop(columns=['city', 'enrollee_id'])
x_test = x_test.drop(columns=['city', 'enrollee_id'])
y = y.drop(columns=['enrollee_id'])

data = pd.concat([X, y], axis=1)

# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()

# X['training_hours'] = scaler.fit_transform(X[['training_hours']])
# x_test['training_hours'] = scaler.transform(x_test[['training_hours']])

##

from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=10, random_state=42)

##

from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

model_xgb = XGBClassifier(n_estimators=100, max_depth=5, eval_metric='error', random_state=42)
model_xgb.fit(x_train, y_train)

y_val_pred = model_xgb.predict(x_val)
print('xgboost roc_auc_score : ', roc_auc_score(y_val, y_val_pred))

from sklearn.ensemble import RandomForestClassifier

model_rfc = RandomForestClassifier(n_estimators=200, max_depth=5, criterion='entropy', random_state=42)
model_rfc.fit(x_train, y_train)

y_val_pred = model_rfc.predict(x_val)

print('randomforest roc_auc_score : ',roc_auc_score(y_val, y_val_pred))

y_test_pred = pd.DataFrame(model_xgb.predict(x_test)).rename(columns={0:'target'})
final = pd.concat([x_test_id, y_test_pred], axis=1)
# final.to_csv('12345.csv', index=False)
final


/Users/dabeen/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/dabeen/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


xgboost roc_auc_score :  0.7083333333333334


<ipython-input-3-11684cedfad1>:85: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_rfc.fit(x_train, y_train)


randomforest roc_auc_score :  0.7083333333333334


,enrollee_id,target
0,7129,0.0
1,31037,0.0
2,22179,0.0
3,29724,0.0
4,17977,0.0
...,...,...
6701,3601,0.0
6702,2745,0.0
6703,18520,0.0
6704,10067,0.0
